# Banking Customer SQL Agent (Indian Context)

This notebook demonstrates how to:
1. Generate dummy banking data with **Indian names, banks, and cities**.
2. Store it in a SQLite database.
3. Use LangChain and OpenAI to query the data using natural language.

In [ ]:
import sqlite3
import random
import pandas as pd
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
model_name ="gpt-5-nano"
llm = ChatOpenAI(model=model_name)

# Load environment variables (ensure OPENAI_API_KEY is set)
load_dotenv("../.env")

## 1. Generate Dummy Data (Indian Context)

In [ ]:
def generate_indian_dummy_data(num_rows=100):
    first_names = ["Aarav", "Vihaan", "Aditya", "Arjun", "Sai", "Rohan", "Ishaan", "Riya", "Ananya", "Diya", "Saanvi", "Priya", "Neha", "Sneha", "Rahul", "Amit", "Vikram", "Pooja", "Kavita", "Suresh"]
    last_names = ["Sharma", "Patel", "Kumar", "Singh", "Gupta", "Rao", "Reddy", "Nair", "Verma", "Mehta", "Iyer", "Joshi", "Desai", "Malhotra", "Bhat", "Saxena", "Mishra", "Das", "Chopra", "Jain"]
    banks = ["State Bank of India (SBI)", "HDFC Bank", "ICICI Bank", "Axis Bank", "Punjab National Bank (PNB)", "Kotak Mahindra Bank", "Bank of Baroda", "IndusInd Bank"]
    places = ["Mumbai, MH", "Delhi, DL", "Bangalore, KA", "Hyderabad, TG", "Chennai, TN", "Kolkata, WB", "Pune, MH", "Ahmedabad, GJ", "Jaipur, RJ", "Lucknow, UP"]
    spending_categories = ["Groceries", "Travel", "Electronics", "Utilities", "Dining", "Healthcare", "Education", "Fuel"]

    data = []
    for _ in range(num_rows):
        customer_name = f"{random.choice(first_names)} {random.choice(last_names)}"
        account_number = ''.join([str(random.randint(0, 9)) for _ in range(12)]) # 12 digits common in India
        bank_name = random.choice(banks)
        monthly_average_balance = round(random.uniform(5000.0, 200000.0), 2) # In INR
        yearly_income = round(random.uniform(300000.0, 3000000.0), 2) # In INR
        place = random.choice(places)
        top_spending_category = random.choice(spending_categories)

        data.append((customer_name, account_number, bank_name, monthly_average_balance, yearly_income, place, top_spending_category))
    
    return data

data_indian = generate_indian_dummy_data(100)
# Show first 5 rows to verify
for row in data_indian[:5]:
    print(row)

## 2. Create SQL Database & Table

In [ ]:
# Connect to SQLite database (creates file if not exists)
db_name_indian = "banking_customers_indian.db"
conn = sqlite3.connect(db_name_indian)
cursor = conn.cursor()

# Create table
create_table_query = """
CREATE TABLE IF NOT EXISTS consumers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_name TEXT,
    account_number TEXT,
    bank_name TEXT,
    monthly_average_balance REAL,
    yearly_income REAL,
    place TEXT,
    top_spending_category TEXT
);
"""
cursor.execute(create_table_query)

# Clear existing data to avoid duplicates if re-run
cursor.execute("DELETE FROM consumers")

# Insert data
insert_query = """
INSERT INTO consumers (customer_name, account_number, bank_name, monthly_average_balance, yearly_income, place, top_spending_category)
VALUES (?, ?, ?, ?, ?, ?, ?)
"""
cursor.executemany(insert_query, data_indian)
conn.commit()

print(f"Inserted {cursor.rowcount} rows into 'consumers' table.")

conn.close()

## 3. Query with LangChain SQL Agent (Custom Prompt)

In [ ]:


# Define Custom System Prompt
prompt_template = """
You are an expert banking assistant designed to interact with a SQL database containing customer financial data.
Your goal is to help users understand customer demographics, spending habits, and financial status.

The database has a table named `consumers` with the following columns:
- `customer_name`: Name of the customer.
- `account_number`: Unique identifier for the account.
- `bank_name`: Name of the bank where the account is held (e.g., SBI, HDFC).
- `monthly_average_balance`: Customer's average monthly balance in INR.
- `yearly_income`: Customer's annual income in INR.
- `place`: City and State of the customer (e.g., Mumbai, MH).
- `top_spending_category`: The category where the customer spends the most.

Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

Rules:
1. You can order the results by relevant columns (`yearly_income`, `monthly_average_balance`) to return interesting examples.
2. Never query for all columns from a specific table, only ask for the relevant columns given the question.
3. You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
4. DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
5. If asked about "rich" customers, assume it refers to high `yearly_income` or `monthly_average_balance`.

To start you should ALWAYS look at the tables in the database to see what you can query. Do NOT skip this step.
Then you should query the schema of the most relevant tables.
"""

system_prompt = prompt_template.format(dialect=db.dialect, top_k=5)

In [ ]:

from langchain.agents import create_agent


agent = create_agent(
    model=llm,
    tools,
    system_prompt=system_prompt,
)

## 4. Run Natural Language Queries (Indian Context)

In [ ]:
query1 = "Which customer in Mumbai has the highest yearly income?"
response1 = agent_executor.invoke(query1)
print(response1['output'])

In [ ]:
query2 = "What is the average monthly balance of HDFC Bank customers?"
response2 = agent_executor.invoke(query2)
print(response2['output'])

In [ ]:
query3 = "List the names of customers who spend most on 'Groceries' and live in Bangalore."
response3 = agent_executor.invoke(query3)
print(response3['output'])